# Otic Mesenchyme and Sensory Epithelium Cell-Cell Communication Analysis

Prepared by: Abel P. David (Northwestern University)

Date: 11/6/2025

Prepared for: Ippei Kishimoto and Alan Cheng (Stanford University)

In [ ]:
library(CellChat)
library(patchwork)
library(NMF)
library(circlize)
library(ComplexHeatmap)
library(Seurat)
library(Matrix)
library(magrittr)
library(tidyverse)
library(SingleCellExperiment)
library(remotes)
library(renv)
library(future)
library(furrr)
library(SeuratDisk)
library(devtools)
library(Seurat)
library(SeuratDisk)

# CellChat - E13/E14 Separated

[Github: jinworks/CellChat](https://github.com/jinworks/CellChat)

- Communication Probability calculation with `truncatedMean` method and `trim = 0.1`


In [ ]:
# Modify the prefix of the figure output files
data_name = "E13E14_PSC_SeparatedV2"
so <- readRDS("e13_e14_harmony_separated.rds")

In [ ]:
DimPlot(so, group.by = "final_cluster")

In [ ]:
pdf(paste0(data_name, "_UMAP.pdf"), width = 6, height = 5, family = "Helvetica")
DimPlot(so, group.by = "final_cluster")
dev.off()

In [ ]:
# CellChat Accepts the normalized data from Seurat
so <- NormalizeData(so)
Idents(so) <- "final_cluster"
labels <- Idents(so)
assay <- "RNA"
data.input <- GetAssayData(so, assay = assay, layer = "data")
meta <- data.frame(group = labels, row.names = names(labels))

# Create CellChat Object
cellchat <- createCellChat(object = data.input, meta = meta, group.by = "group")

# Check Cell Information
levels(cellchat@idents)
cellchat@idents %>% table()

# Set the Ligand-Receptor interaction database: CellChatDB.mouse
CellChatDB <- CellChatDB.mouse
showDatabaseCategory(CellChatDB)

## Show the structure of the CellChatDB database
dplyr::glimpse(CellChatDB$interaction)
CellChatDB.use <- CellChatDB

## Set the use database in the object
cellchat@DB <- CellChatDB.use

In [ ]:
# Preprocessing expression data for cell-cell communication analysis
cellchat <- subsetData(cellchat)
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)

# Project gene expression data onto PPI network
cellchat <- smoothData(cellchat, adj = PPI.mouse) # mouse PPI network
cellchat <- computeCommunProb(cellchat, type = "truncatedMean", trim = 0.1)

# Filter out the cell-cell communication if there are only few number of cells in certain cell groups
cellchat <- filterCommunication(cellchat, min.cells = 10) # This is is the filter step.

# Extracting the inferred cellular communication network as a dataframe
df.net <- subsetCommunication(cellchat)

# Infer the cell cell signaling at a pathway level
cellchat <- computeCommunProbPathway(cellchat)

# Calculate the aggregated cell-cell communication network
cellchat <- aggregateNet(cellchat)

In [ ]:
# Check trim thresholds
computeAveExpr(cellchat, features = c("Wnt5a","Wnt7b"), type = "truncatedMean", trim = 0.1)

## Modify Color Scheme

In [ ]:
scPalette <- function(n) {
colorSpace <- c("#ED83B5", "#C0A1CB","#DD4477", "#F09732")
  if (n <= length(colorSpace)) {
    colors <- colorSpace[1:n]
  } else {
    colors <- grDevices::colorRampPalette(colorSpace)(n)
  }
  return(colors)
}

ns <- asNamespace("CellChat")
# 1) unlock just the binding you need
rlang::env_binding_unlock(ns, "scPalette")

# 2) rebind the value (prefer env_bind over assign)
rlang::env_bind(ns, scPalette = scPalette)

# 3) re-lock that binding
rlang::env_binding_lock(ns, "scPalette")
# Select Subset of cells: including supporitn cells

In [ ]:
extractEnrichedLR(cellchat, signaling = cellchat@netP$pathways, geneLR.return = FALSE)
cellchat@netP$pathways

## Chord Plot

In [ ]:
pathway <- c("WNT", "ncWNT")
print(paste("Plotting pathway:", pathway))
pdf(paste0(data_name, "_chord.pdf"), width = 8, height = 8, family = "Helvetica")
netVisual_chord_gene(
  cellchat, 
  signaling = pathway,
  targets.use = c(3,4),
  sources.use = c(1,2),
  small.gap = 1,
  big.gap = 10,
  link.target.prop = TRUE
)
dev.off()


## Circle Plot

In [ ]:
pathway <- c("WNT", "ncWNT")
pdf(paste0(data_name, "_circle.pdf"), width = 8, height = 8, family = "Helvetica")
netVisual_aggregate(
  cellchat, 
  signaling = pathway, 
  layout = "circle"
)
dev.off()

## Contribution Bar Plot

In [ ]:
pdf(paste0(data_name, "_contribution_bar.pdf"), width = 8, height = 8, family = "Helvetica")
netAnalysis_contribution(
  cellchat, 
  signaling = pathway,
  targets.use = c(3,4),
  sources.use = c(1,2)
)
dev.off()